# Loading datasets and model

In [ ]:
# Download the ESM-2 model before running
tokenizer_model = "./model_esm2/esm2_t33_650M_UR50D"
base_model = "./model_esm2/esm2_t33_650M_UR50D"

string_organism = "B. pertussis"
organism = "bpertussis"

path_results = "./results_esm2/"
path_models = "./models_esm2/"

method = "epitopetransfer"
method_name = "EpitopeTransfer"

# generating fasta path for all folds
train_fasta_folds = ["fold1", "fold2", "fold4"]
test_fasta_fold = "fold5"
train_fasta_files = [f"./fasta/folds/{organism}/{fold}.fasta" for fold in train_fasta_folds]
test_fasta_file = f"./fasta/folds/{organism}/{test_fasta_fold}.fasta"

#folds
fold_base_path = f"./input/organismos/{organism}/folds/"
train_fold_paths = [f"{fold_base_path}fold{i}.csv" for i in [1, 2, 4]]

test_fold = "fold5"
test_path = f"{fold_base_path}{test_fold}.csv"

base_path_predictions = f"./predictions_esm2/{method}/{organism}"

path_predictions = f"{base_path_predictions}/{test_fold}.csv"
fold_numbers = 4 # number of train folds + test_fold
max_seq_length = 1024
overlap = 256

In [ ]:
!pip install transformers
!pip install fair-esm  # latest release, OR:
!pip install git+https://github.com/facebookresearch/esm.git
!pip install biopython
!pip install optuna-integration

In [ ]:
from Bio import SeqIO
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np

def predict_fasta_sequences(fasta_file):


    model = AutoModel.from_pretrained(base_model)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, do_lower_case=False)

    fasta_sequences = SeqIO.parse(open(fasta_file), 'fasta')
    sequence_dict = {}

    model.eval()  # Disable dropout

    for record in fasta_sequences:
        sequence = str(record.seq)
        sequence_length = len(sequence)
        embeddings = None

        if sequence_length > max_seq_length:
            # Divede the sequendo in chunks with overlap
            for i in range(0, sequence_length, max_seq_length - overlap):
                chunk = sequence[i:i + max_seq_length]
                if len(chunk) < max_seq_length:
                    chunk = chunk + ' ' * (max_seq_length - len(chunk))  # Padding if less than max_seq_length
                features = tokenizer.batch_encode_plus(
                    [chunk],
                    add_special_tokens=True,
                    padding='max_length',
                    max_length=max_seq_length,
                    truncation=True,
                    return_tensors='pt',
                    return_attention_mask=True
                )

                with torch.no_grad():
                    outputs = model(features['input_ids'], features['attention_mask'])

                last_hidden_state = outputs[0]
                array_last_hidden_state = last_hidden_state.detach().numpy()

                if embeddings is None:
                    embeddings = array_last_hidden_state[0]
                else:
                    embeddings = np.concatenate((embeddings, array_last_hidden_state[0][overlap:]))
        else:
            features = tokenizer.batch_encode_plus(
                [sequence],
                add_special_tokens=True,
                padding='max_length',
                max_length=max_seq_length,
                truncation=True,
                return_tensors='pt',
                return_attention_mask=True
            )

            with torch.no_grad():
                outputs = model(features['input_ids'], features['attention_mask'])

            last_hidden_state = outputs[0]
            embeddings = last_hidden_state.detach().numpy()[0]

        sequence_dict[record.id] = {
            'completed_sequence': sequence,
            'embeddings': embeddings
        }

    return sequence_dict

In [ ]:
import pandas as pd

# Select labeled regions after forward pass through the neural network
def select_labeled_regions(dataset, sequence_dict):
    list_amino_features = []
    list_amino_label = []

    for idx, row in dataset.iterrows():
        protein_id = row["Info_protein_id"]

        if protein_id not in sequence_dict:
            print(f"{protein_id} is not in sequence_dict")
            continue

        data = sequence_dict[protein_id]
        sequence_slice = data['embeddings']
        start = row["Info_start_pos"]
        end = row["Info_end_pos"]
        label = row["Class"]
        sequence = data['completed_sequence']

        for index_amino in range(start - 1, end):
            list_amino_features.append(sequence_slice[index_amino])
            list_amino_label.append(label)

    return  list_amino_features, list_amino_label

In [ ]:
# Function to combine the results of multiple calls to predict_fasta_sequences
def predict_fasta_sequences_multiple(files):
    combined_sequence_dict = {}
    for fasta_file in files:

        print(fasta_file)
        sequence_dict = predict_fasta_sequences(fasta_file)
        combined_sequence_dict.update(sequence_dict)
    return combined_sequence_dict

train_sequence_dict = predict_fasta_sequences_multiple(train_fasta_files)
test_sequence_dict = predict_fasta_sequences(test_fasta_file)

In [ ]:
import pandas as pd

# Process each training fodl
for i, fold_path in enumerate(train_fold_paths, start=1):

    df_fold = pd.read_csv(fold_path)


    processed_data = select_labeled_regions(df_fold, train_sequence_dict)

    # Convertendo as listas em DataFrames
    features_df = pd.DataFrame(processed_data[0])
    labels_df = pd.DataFrame(processed_data[1])

    # Salvando features e labels de cada fold em arquivos CSV
    features_csv_path = f"{fold_base_path}{method}/processed_fold{i}_features.csv"
    labels_csv_path = f"{fold_base_path}{method}/processed_fold{i}_labels.csv"

    os.makedirs(os.path.dirname(features_csv_path), exist_ok=True)
    os.makedirs(os.path.dirname(labels_csv_path), exist_ok=True)

    features_df.to_csv(features_csv_path, index=False)
    labels_df.to_csv(labels_csv_path, index=False)

    print(f"Processed fold{i} features saved to {features_csv_path}")
    print(f"Processed fold{i} labels saved to {labels_csv_path}")


df_test = pd.read_csv(test_path)

dados_processados = select_labeled_regions(df_test, test_sequence_dict)

features_csv_path = f"{fold_base_path}{method}/processed_test_features.csv"
labels_csv_path = f"{fold_base_path}{method}/processed_test_labels.csv"
pd.DataFrame(dados_processados[0]).to_csv(features_csv_path, index=False)
pd.DataFrame({"Label": dados_processados[1]}).to_csv(labels_csv_path, index=False)

print(f"Processed test features saved to {features_csv_path}")
print(f"Processed test labels saved to {labels_csv_path}")


In [ ]:
import pandas as pd

loaded_data = {}

# Loading the traing datasets
for i in range(1, fold_numbers):
    features_csv_path = f"{fold_base_path}{method}/processed_fold{i}_features.csv"
    labels_csv_path = f"{fold_base_path}{method}/processed_fold{i}_labels.csv"

    loaded_data[f"fold{i}"] = {
        "features": pd.read_csv(features_csv_path),
        "labels": pd.read_csv(labels_csv_path)
    }

# Loading the test datasets
test_features_csv_path = f"{fold_base_path}{method}/processed_test_features.csv"
test_labels_csv_path = f"{fold_base_path}{method}/processed_test_labels.csv"

loaded_data["test"] = {
    "features": pd.read_csv(test_features_csv_path),
    "labels": pd.read_csv(test_labels_csv_path)
}



In [ ]:
import numpy as np
import pandas as pd
import optuna
from optuna.distributions import IntDistribution, CategoricalDistribution
from optuna.integration.sklearn import OptunaSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, make_scorer, roc_auc_score, f1_score
import pickle
import os


def perform_validation_holdout(loaded_data):
    X = loaded_data["fold1"]["features"]
    y = loaded_data["fold1"]["labels"].iloc[:, 0]

    final_model = RandomForestClassifier()  # default hyperparameters
    final_model.fit(X, y)

    best_threshold = 0.5

    # Save final model
    os.makedirs(path_models, exist_ok=True)
    model_path = os.path.join(path_models, f"{method}_{organism}.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(final_model, f)
    print(f"Final model saved to {model_path}")

    return final_model, best_threshold, final_model.get_params()

def custom_mcc(estimator, X, y):
    preds = estimator.predict(X)
    return matthews_corrcoef(y, preds)


def perform_cross_validation(loaded_data):

    all_features = pd.concat([loaded_data[f"fold{i}"]["features"] for i in range(1, len(loaded_data))], ignore_index=True)
    all_labels = pd.concat([loaded_data[f"fold{i}"]["labels"] for i in range(1, len(loaded_data))], ignore_index=True).iloc[:, 0]

    fold_sizes = [len(loaded_data[f"fold{i}"]["features"]) for i in range(1, len(loaded_data))]

    # Generates sequential indices for concatenated folds
    fold_indices = []
    current = 0
    for size in fold_sizes:
        fold_indices.append(list(range(current, current + size)))
        current += size

    # Defines training and validation indices based on sequential indices
    train_indices_list = []
    val_indices_list = []

    for i in range(len(fold_indices)):
        val_indices = fold_indices[i]
        train_indices = [idx for j in range(len(fold_indices)) if j != i for idx in fold_indices[j]]
        val_indices_list.append(val_indices)
        train_indices_list.append(train_indices)


    # Define a custom class for StratifiedKFold
    class CustomStratifiedKFold:
        def __init__(self, train_indices_list, val_indices_list):
            self.train_indices_list = train_indices_list
            self.val_indices_list = val_indices_list
            self.n_splits = len(train_indices_list)

        def split(self, X, y=None, groups=None):
            for i in range(self.n_splits):
                yield self.train_indices_list[i], self.val_indices_list[i]

        def get_n_splits(self, X=None, y=None, groups=None):
            return self.n_splits

    custom_skf = CustomStratifiedKFold(train_indices_list, val_indices_list)

    # Hyperparameters to be optimized by OptunaSearchCV
    param_distributions = {
        'n_estimators': IntDistribution(low=100, high=500),
        'max_depth': CategoricalDistribution(choices=[None, 10, 20, 30]),
        'min_samples_split': IntDistribution(low=2, high=10),
        'min_samples_leaf': IntDistribution(low=1, high=10),
        'max_features': CategoricalDistribution(choices=['sqrt', 'log2', None]),
        'bootstrap': CategoricalDistribution(choices=[True, False]),
        'criterion': CategoricalDistribution(choices=['gini', 'entropy'])
    }

    model = RandomForestClassifier(random_state=42, n_jobs=-1)
    optuna_search = OptunaSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_trials=100,
        random_state=42,
        cv=custom_skf,
        n_jobs=-1,
        scoring=custom_mcc
    )

    # Perform search for the best hyperparameters using custom StratifiedKFold
    optuna_search.fit(all_features, all_labels)

    # Get the best hyperparameters
    best_hyperparams = optuna_search.best_params_
    final_model = RandomForestClassifier(**best_hyperparams, random_state=42, n_jobs=-1)

    final_model.fit(all_features, all_labels)

    # Get the prediction probabilities for all data
    all_probs = final_model.predict_proba(all_features)[:, 1]

    # Determine the best threshold by maximizing the MCC
    thresholds = np.arange(0, 1, 0.001)

    mcc_scores = [matthews_corrcoef(all_labels, all_probs >= t) for t in thresholds]
    mcc_best_threshold_index = np.argmax(mcc_scores)
    mcc_best_threshold = thresholds[mcc_best_threshold_index]

    # Save the best model
    model_path = os.path.join(path_models, f"{method}_{organism}.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(final_model, f)
    print(f"Best model saved to {model_path}")

    return final_model, mcc_best_threshold, best_hyperparams

def evaluate_model_on_test(test_features, test_labels, model, mcc_threshold):

    test_probs = model.predict_proba(test_features)[:, 1]
    predictions_mcc = test_probs >= mcc_threshold
    auc = roc_auc_score(test_labels, test_probs)
    f1_at_mcc_threshold = f1_score(test_labels, predictions_mcc)
    mcc_at_mcc_threshold = matthews_corrcoef(test_labels, predictions_mcc)

    print(f"AUC: {auc}")
    print(f"At MCC Threshold -> F1 Score: {f1_at_mcc_threshold}, MCC: {mcc_at_mcc_threshold}")

    results = pd.DataFrame([{
        "Method": method,
        "Organism": organism,
        "Test probabilities": str(test_probs.tolist()),
        "MCC threshold": mcc_threshold
    }])

    results_file_path = os.path.join(path_results, f"{method}_{organism}_test_prob_thresholds.csv")

    if os.path.exists(results_file_path):
        existing_results = pd.read_csv(results_file_path)
        results = pd.concat([existing_results, results], ignore_index=True)

    # Saving the results to a csv
    results.to_csv(results_file_path, index=False)

    return {
        'auc': auc,
        'mcc_threshold': {'f1': f1_at_mcc_threshold, 'mcc': mcc_at_mcc_threshold}
    }


def main(loaded_data, organism, method, path_results):
    if len(loaded_data) > 2:
        best_model, mcc_best_threshold, best_hyperparams = perform_cross_validation(loaded_data)

        # Loading test datasets
        test_features = loaded_data["test"]["features"]
        test_labels = loaded_data["test"]["labels"].iloc[:, 0]

        # Evaluating the model on the test set
        results_dict = evaluate_model_on_test(test_features, test_labels, best_model, mcc_best_threshold)

        results = pd.DataFrame({
            "Dataset":        [organism] * 3,
            "Method":         [method]   * 3,
            "Metric":         ["AUC",
                              "F1 (MCC Threshold)",
                              "MCC (MCC Threshold)"],
            "Value":          [results_dict['auc'],
                              results_dict['mcc_threshold']['f1'],
                              results_dict['mcc_threshold']['mcc']],
            "Best threshold": ["",
                              mcc_best_threshold,
                              mcc_best_threshold]
        })


        results["Value"] = results["Value"].round(3)

        # Including the best hyperparameters as a single string
        best_hyperparams_str = ', '.join([f"{key}: {value}" for key, value in best_hyperparams.items()])
        results["Best hyperparameters"] = [""] * len(results)
        results.loc[0, "Best hyperparameters"] = best_hyperparams_str

        results_file_path = os.path.join(path_results, "test_metrics_summary.csv")

        # Checking if the file already exists and loading existing content if necessary
        if os.path.exists(results_file_path):
            existing_results = pd.read_csv(results_file_path)
            results = pd.concat([existing_results, results], ignore_index=True)

        results.to_csv(results_file_path, index=False)

        print(f"Results saved to {results_file_path}")
    else:
        best_model, mcc_best_threshold, best_hyperparams = perform_validation_holdout(loaded_data)

        test_features = loaded_data["test"]["features"]
        test_labels = loaded_data["test"]["labels"].iloc[:, 0]

        results_dict = evaluate_model_on_test(test_features, test_labels, best_model, mcc_best_threshold)

        results = pd.DataFrame({
            "Dataset":        [organism] * 3,
            "Method":         [method]   * 3,
            "Metric":         ["AUC",
                              "F1 (MCC Threshold)",
                              "MCC (MCC Threshold)"],
            "Value":          [results_dict['auc'],
                              results_dict['mcc_threshold']['f1'],
                              results_dict['mcc_threshold']['mcc']],
            "Best threshold": ["",
                              mcc_best_threshold,
                              mcc_best_threshold]
        })


        results["Value"] = results["Value"].round(3)

        best_hyperparams_str = ', '.join([f"{key}: {value}" for key, value in best_hyperparams.items()])
        results["Best hyperparameters"] = [""] * len(results)
        results.loc[0, "Best hyperparameters"] = best_hyperparams_str

        results_file_path = os.path.join(path_results, "test_metrics_summary.csv")

        if os.path.exists(results_file_path):
            existing_results = pd.read_csv(results_file_path)
            results = pd.concat([existing_results, results], ignore_index=True)

        results.to_csv(results_file_path, index=False)

        print(f"Results saved to {results_file_path}")

main(loaded_data, organism, method, path_results)
